# MongoDB Part 2 Exercises

## 1: Working with Distinct Values, $elemMatch, and Regex

In [ ]:
from pymongo import MongoClient

client = MongoClient()
db = client.nobel

### Never from there, but sometimes there at last

There are some recorded countries of death (`"diedCountry"`) that do not appear as a country of birth (`"bornCountry"`) for laureates. One such country is "East Germany".

- Return a set of all such countries as `countries`.

In [ ]:
# Countries recorded as countries of death but not as countries of birth
countries = set(____) - set(____)
print(countries)

### Countries of affiliation

We saw in the last exercise that countries can be associated with a laureate as their country of birth and as their country of death. For each prize a laureate received, they may also have been affiliated with an institution at the time, located in a country.

- Determine the number of distinct countries recorded as part of an affiliation for laureates' prizes. Save this as `count`.

In [ ]:
# The number of distinct countries of laureate affiliation for prizes
count = ____(db.laureates.____(____))
print(count)

### Triple plays (mostly) all around

All prize categories but one – literature – have had prizes shared by three or more laureates.

- Save a filter document `criteria` that, when passed to `db.prizes.distinct`, returns all prize categories shared by three or more laureates. That is, `"laureates.2"` must exist for such documents.
- Save these prize categories as a Python `set` called `triple_play_categories`.
- Confirm via an assertion that "literature" is the only prize category with no prizes shared by three or more laureates.

In [ ]:
# Save a filter for prize documents with three or more laureates
criteria = {____: {____: ____}}

# Save the set of distinct prize categories in documents satisfying the criteria
triple_play_categories = ____(db.prizes.distinct(____, criteria))

# Confirm literature as the only category not satisfying the criteria.
assert set(db.prizes.distinct(____)) - triple_play_categories == {____}

### Other sharing after World War II

Similar to the example in the lesson, what is the approximate ratio of the number of laureates who won an *unshared* prize in categories other than physics, chemistry, and medicine after World War II to the number of laureates who won a *shared* prize in categories other than physics, chemistry, and medicine after World War II?

- Save an `$elemMatch` filter `unshared` to count laureates with unshared prizes in categories other than ("not in") `["physics", "chemistry", "medicine"]` in or after 1945.
- Save an `$elemMatch` filter `shared` to count laureates with shared (i.e., "share" is not "1") prizes in categories other than `["physics", "chemistry", "medicine"]` in or after 1945.



In [ ]:
# Save a filter for laureates with unshared prizes
unshared = {
    "prizes": {____: {
        ____: {____: ["physics", "chemistry", "medicine"]},
        "share": "1",
        "year": {____: "1945"},
    }}}

# Save a filter for laureates with shared prizes
shared = {
    "prizes": {____: {
        ____: {____: ["physics", "chemistry", "medicine"]},
        "share": {____: "1"},
        "year": {____: "1945"},
    }}}

ratio = db.laureates.____(____) / db.laureates.____(____)
print(ratio)

### Organizations and prizes over time

How many organizations won prizes before 1945 versus in or after 1945?

- Save a filter `before` to count organization laureates with prizes won before 1945. Recall that organization status is encoded with the "gender" field, and that dot notation is needed to access a laureate's "year" field within its "prizes" array.
- Save a filter `in_or_after` to count organization laureates with prizes won in or after 1945.

In [ ]:
# Save a filter for organization laureates with prizes won before 1945
before = {
    ____: ____,
    ____: {____: "1945"},
    }

# Save a filter for organization laureates with prizes won in or after 1945
in_or_after = {
    ____: ____,
    ____: {____: "1945"},
    }

n_before = db.laureates.count_documents(before)
n_in_or_after = db.laureates.count_documents(in_or_after)
ratio = n_in_or_after / (n_in_or_after + n_before)
print(ratio)

### Germany, then and now

Just as we saw with Poland, there are laureates who were born somewhere that was in Germany at the time but is now not, and others born somewhere that was not in Germany at the time but now is.

- Use a regular expression object to filter for laureates with "Germany" in their "bornCountry" value.

In [ ]:
# Filter for laureates with "Germany" in their "bornCountry" value
criteria = {"bornCountry": {"$regex": ____}}
print(set(db.laureates.distinct("bornCountry", criteria)))

- Use a regular expression object to filter for laureates with a "bornCountry" value starting with "Germany".

In [ ]:
# Filter for laureates with a "bornCountry" value starting with "Germany"
criteria = {"bornCountry": {____:____}}
print(set(db.laureates.distinct("bornCountry", criteria)))

- Use a regular expression object to filter for laureates born in what was at the time Germany but is now another country.

In [ ]:
# Fill in a string value to be sandwiched between the strings "^" and "now"
criteria = {"bornCountry": {____:"^" + ____ + "now"}}
print(set(db.laureates.distinct("bornCountry", criteria)))

- Use a regular expression object to filter for laureates born in what is now Germany but at the time was another country.

In [ ]:
# Filter for currently-Germany countries of birth.
# Fill in a string value to be sandwiched between the strings "now" and "$"
criteria = {"bornCountry": {____:"now" + ____ + "$"}}
print(set(db.laureates.distinct("bornCountry", criteria)))

### The prized transistor

Three people shared a Nobel prize "for their researches on semiconductors and their discovery of the transistor effect". We can filter on "transistor" as a substring of a laureate's "prizes.motivation" field value to find these laureates.

- Save a filter `criteria` that finds laureates with `prizes.motivation` values containing "transistor" as a substring. The substring can appear anywhere within the value, so no anchoring characters are needed.
- Save to `first` and `last` the field names corresponding to a laureate's first name and last name (i.e. "surname") so that we can print out the names of these laureates.



In [ ]:
# Save a filter for laureates with prize motivation values containing "transistor" as a substring
criteria = {____: {____:____}}

# Save the field names corresponding to a laureate's first name and last name
first, last = ____, ____
print([(laureate[first], laureate[last]) for laureate in db.laureates.find(criteria)])

## 2: Projection and Sorting

### Rounding up the G.S. crew

We can use the regular expression operator `$regex` to find laureates whose initials are G.S. Let's use projection and list comprehension to collect the full names of these laureates by concatenating their first ("firstname") and last ("surname") names.

- Fill in the blanks to save a list `names` of full names ("firstname" plus "surname") of laureates with initials G.S. (ignoring middles names/initials). You'll need to both filter on names as well as project out the fields required to collect the full names.


In [ ]:
# Collect a list of full names
names = [" ".join([doc[____], doc[____]])
         for doc in db.laureates.find(
             {____: {"$regex": "^G"},
              ____: {"$regex": "^S"}},
             {____: 1, ____: 1})]
print(names)

### Sorting together: MongoDB + Python

You will print out the names of all physics laureates, with one line printed for each award year, in chronological order. Each line will list laureates for that year in alphabetical order by surname ("last" name).

I encourage you to print intermediate results and understand the nested structure of prize documents.

- Construct a sort specification `sort_spec` to fetch physics prizes by ascending year.

In [ ]:
from operator import itemgetter

# Sort by ascending year
sort_spec = [(____, ____)]

- Use `<collection>.find` to construct a `cursor` that fetches prizes with a "category" of "physics", sorts by ascending year, and projects the year and first laureate full name (`laureates.firstname` and `laureates.surname`). *You should encounter an error at year 1916*.

In [ ]:
# Construct a cursor over physics prizes
cursor = db.prizes.____({____: ____}, 
                        {"year": 1, "laureates.firstname": 1, "laureates.surname": 1},
                        sort=sort_spec)

for doc in cursor:
    print("{year}: {first_laureate_firstname} {first_laureate_surname}".format(
        year=doc["year"],
        first_laureate_firstname=doc["laureates"][0]["firstname"],
        first_laureate_surname=doc["laureates"][0]["surname"]))
cursor.rewind() # Rewind cursor to reuse in the next step

- The error is caused by the fact that the Nobel Prize in physics was not awarded in 1916 due to World War I. Supplement the cursor's filter to avoid the error:

In [ ]:
# Construct a fixed cursor over physics prizes
cursor = db.prizes.____({____: ____, ____: {____:True}}, 
                        {"year": 1, "laureates.firstname": 1, "laureates.surname": 1},
                        sort=sort_spec)

for doc in cursor:
    print("{year}: {first_laureate_firstname} {first_laureate_surname}".format(
        year=doc["year"],
        first_laureate_firstname=doc["laureates"][0]["firstname"],
        first_laureate_surname=doc["laureates"][0]["surname"]))
cursor.rewind() # Rewind cursor to reuse in the next step

- Complete the definition of the function `names` so that, given a prize document, it returns a list of formatted names, sorted by ascending "surname", for each of the "laureates" in that prize document.



In [ ]:
# Define a function names() to return a list of formatted names
def names(doc):
    formatted_names = ["{firstname} {surname}".format(**laureate)
          for laureate in sorted(doc[____], key=itemgetter(____))]
    return formatted_names

lines = ["{year}: {names}".format(year=doc["year"], names=" and ".join(names(doc)))
         for doc in cursor]
for line in lines: print(line)

### Gap years

As we saw above, there have been years for which prizes in one or more of the original categories were not awarded.

Sorting first by reverse chronological order and second by alphabetical order of category, collect and format prize documents to produce one formatted entry per year listing categories missing for that year.

- Construct a set `original_categories` of prize categories awarded in 1901.

In [ ]:
import itertools
from operator import itemgetter

# Save the set of prize categories awarded in 1901
original_categories = set(db.prizes.____("category", {____: "1901"}))
print(original_categories)

- Use `<collection>.find` to construct a cursor that yields prize documents only for categories in the list of original categories which contain the `laureates` key and thus were awarded, sorted first by decreasing year and second by increasing category.



In [ ]:
# Construct a cursor over original-category prizes
cursor = db.prizes.____({"category": {____: list(original_categories)}, ____: {____:____}},
                        {"category": 1, "year": 1},
                        sort=[(____, ____), (____, ____)])

- Collect a list `not_awarded` of entries to be printed, one per line, that displays a year and the categories missing for that year. You will collect "category" values for each year and set-subtract them from the original categories.



In [ ]:
# Collect entries for missing prize categories
not_awarded = []
for key, group in itertools.groupby(cursor, key=itemgetter("year")):
    year_categories = set(prize[____] for prize in group)
    missing = ", ".join(sorted(____ - ____))
    if missing: not_awarded.append("{}: {}".format(key, missing))

for line in not_awarded: print(line)

## 3: Indexing

### An index for high-share categories

We want to speed up the following operation:
```python
db.prizes.distinct("category", {"laureates.share": {"$gt": "3"}})
```
- Confirm that the operation takes approximately 1 ms without an index.

- Specify a compound index model `index_model` to pass to `db.prizes.create_index`.

In [ ]:
index_model = [(____,____), (____,____)]
db.prizes.create_index(index_model)

- Confirm that the execution time is now below 400 µs.

In [ ]:
# Drop the index for consistency
db.prizes.drop_index(index_model)

### Recently single?

A prize might be awarded to a single laureate or to several. For each prize category, report the most recent year that a single laureate -- rather than several -- received a prize in that category.

- Specify an index model `index_model` to pass to `db.prizes.create_index` that speeds up finding prizes by category and sorting results by decreasing year. That is, the model should index first on category (ascending) and second on year (descending).
- Save a string `report` for printing the last single-laureate year for each distinct category, one category per line. To do this, for each distinct prize category, find the latest-year prize of that category with a laureate share of "1".

In [ ]:
# Specify an index model for compound sorting
index_model = [(____, 1), (____, -1)]
db.prizes.create_index(index_model)

# Collect the last single-laureate year for each category
report = ""
for category in sorted(db.prizes.____("category")):
    doc = db.prizes.find_one(
        {____: ____, "laureates.share": "1"},
        sort=[(____, ____)]
    )
    report += "{category}: {year}\n".format(**doc)

print(report)

# Drop the index for consistency
db.prizes.drop_index(index_model)

### Born and affiliated

Some countries are, for one or more laureates, both their country of birth ("bornCountry") and a country of affiliation for one or more of their prizes ("prizes.affiliations.country"). You will find the five countries of birth with the highest counts of such laureates.

- Create an index on country of birth ("bornCountry") for `db.laureates` to ensure efficient gathering of distinct values and counting of documents
- Complete the skeleton dictionary comprehension to construct `n_born_and_affiliated`, the count of laureates as described above for each distinct country of birth.


In [ ]:
from collections import Counter

# Ensure an index on country of birth
db.laureates.create_index([(____, 1)])

# Collect a count of laureates for each country of birth
n_born_and_affiliated = {
    country: db.laureates.count_documents({
        ____: ____,
        "prizes.affiliations.country": ____
    })
    for ____ in db.laureates.distinct("bornCountry")
}

five_most_common = Counter(n_born_and_affiliated).most_common(5)
print(five_most_common)

# Drop the index for consistency
db.laureates.drop_index([(____, 1)])